In [1]:
import pandas as pd
print("Pandas version:", pd.__version__)

Pandas version: 2.3.3


In [2]:
fielding = pd.read_csv("../data/raw/mlb/fielding.csv")
fielding.head()

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,aardsda01,2004,1,SFN,NL,P,11,0.0,32.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,aardsda01,2006,1,CHN,NL,P,45,0.0,159.0,1.0,5.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN
2,aardsda01,2007,1,CHA,AL,P,25,0.0,97.0,2.0,4.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
3,aardsda01,2008,1,BOS,AL,P,47,0.0,146.0,3.0,6.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,aardsda01,2009,1,SEA,AL,P,73,0.0,214.0,2.0,5.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN


In [3]:
fielding.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'POS', 'G', 'GS',
       'InnOuts', 'PO', 'A', 'E', 'DP', 'PB', 'WP', 'SB', 'CS', 'ZR'],
      dtype='object')

In [10]:
# 1. Convert Outs to Innings
fielding['Innings'] = fielding['InnOuts'] / 3

# 2. Aggregate to player-season-position
mlb_def_roles = (
    fielding
    .groupby(['playerID', 'yearID', 'POS'], as_index=False)
    .agg(time_in_role = ('Innings', 'sum'))
    .rename(columns={
        'playerID': 'player_id',
        'yearID': 'season',
        'POS': 'position'
    })
)

# 3. Add "League" column
mlb_def_roles['league'] = 'MLB'

mlb_def_roles.head()

,player_id,season,position,time_in_role,league
0,aardsda01,2004,P,10.666667,MLB
1,aardsda01,2006,P,53.000000,MLB
2,aardsda01,2007,P,32.333333,MLB
3,aardsda01,2008,P,48.666667,MLB
4,aardsda01,2009,P,71.333333,MLB


In [11]:
mlb_def_roles["total_time"] = (
    mlb_def_roles
    .groupby(["player_id", "season"])["time_in_role"]
    .transform("sum")
)

mlb_def_roles["role_share"] = mlb_def_roles["time_in_role"] / mlb_def_roles["total_time"]

(mlb_def_roles
 .groupby(["player_id", "season"])["role_share"]
 .sum()
 .describe()
)

count    115488.000000
mean          0.802300
std           0.398266
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: role_share, dtype: float64

In [13]:
batting = pd.read_csv("../data/raw/mlb/batting.csv")
pitching = pd.read_csv("../data/raw/mlb/pitching.csv")

# -----------------------
# HITTER ROLES (PA-weighted)
# -----------------------
# Making sure missing columns don't break things
for c in ["BB", "HBP", "SH", "SF"]:
    if c not in batting.columns:
        batting[c] = 0

bat_agg = (
    batting
    .groupby(["playerID", "yearID"], as_index=False)
    .agg(AB=("AB", "sum"),
         BB=("BB", "sum"),
         HBP=("HBP", "sum"),
         SH=("SH", "sum"),
         SF=("SF", "sum"))
)

bat_agg["PA"] = bat_agg["AB"] + bat_agg["BB"] + bat_agg["HBP"] + bat_agg["SH"] + bat_agg["SF"]

hitter_roles = (
    bat_agg.loc[bat_agg["PA"] > 0, ["playerID", "yearID", "PA"]]
    .rename(columns={"playerID": "player_id", "yearID": "season", "PA": "time_in_role"})
)
hitter_roles["position"] = "HITTER"
hitter_roles["league"] = "MLB"

hitter_roles = hitter_roles[["player_id", "season", "position", "time_in_role", "league"]]

# -----------------------
# PITCHER ROLES (IP-weighted)
# -----------------------
# Lahman Pitching uses IPouts (outs recorded). Convert to innings.
pit_agg = (
    pitching
    .groupby(["playerID", "yearID"], as_index=False)
    .agg(IPouts=("IPouts", "sum"))
)

pitcher_roles = pit_agg.copy()
pitcher_roles["time_in_role"] = pitcher_roles["IPouts"] / 3.0
pitcher_roles = pitcher_roles.loc[pitcher_roles["time_in_role"] > 0, ["playerID", "yearID", "time_in_role"]]
pitcher_roles = pitcher_roles.rename(columns={"playerID": "player_id", "yearID": "season"})
pitcher_roles["position"] = "P"
pitcher_roles["league"] = "MLB"

pitcher_roles = pitcher_roles[["player_id", "season", "position", "time_in_role", "league"]]

# -----------------------
# COMBINE
# -----------------------
mlb_roles = pd.concat([mlb_def_roles, hitter_roles, pitcher_roles], ignore_index=True)

mlb_roles.head(), hitter_roles.head(), pitcher_roles.head()

(   player_id  season position  time_in_role league  total_time  role_share
 0  aardsda01    2004        P     10.666667    MLB   10.666667         1.0
 1  aardsda01    2006        P     53.000000    MLB   53.000000         1.0
 2  aardsda01    2007        P     32.333333    MLB   32.333333         1.0
 3  aardsda01    2008        P     48.666667    MLB   48.666667         1.0
 4  aardsda01    2009        P     71.333333    MLB   71.333333         1.0,
     player_id  season position  time_in_role league
 1   aardsda01    2006   HITTER           3.0    MLB
 3   aardsda01    2008   HITTER           1.0    MLB
 8   aardsda01    2015   HITTER           1.0    MLB
 9   aaronha01    1954   HITTER         509.0    MLB
 10  aaronha01    1955   HITTER         665.0    MLB,
    player_id  season position  time_in_role league
 0  aardsda01    2004        P     10.666667    MLB
 1  aardsda01    2006        P     53.000000    MLB
 2  aardsda01    2007        P     32.333333    MLB
 3  aardsda01   

In [ ]:
import numpy as np

def entropy(shares):
    shares = shares[shares > 0]
    return -np.sum(shares * np.log(shares))

utility_entropy = (
    mlb_roles
    .groupby(["player_id", "season"])
    .agg(
        utility_entropy=("role_share", entropy),
        n_roles=("position", "nunique")
    )
    .reset_index()
)

utility_entropy["max_entropy"] = np.log(utility_entropy["n_roles"])
utility_entropy["utility_score"] = (
    utility_entropy["utility_entropy"] / utility_entropy["max_entropy"]
)

# handle n_roles = 1
utility_entropy.loc[utility_entropy["n_roles"] == 1, "utility_score"] = 0.0


utility_entropy.head()

,player_id,season,utility_entropy,n_roles,max_entropy,utility_score
0,aardsda01,2004,-0.0,1,0.000000,0.0
1,aardsda01,2006,-0.0,2,0.693147,-0.0
2,aardsda01,2007,-0.0,1,0.000000,0.0
3,aardsda01,2008,-0.0,2,0.693147,-0.0
4,aardsda01,2009,-0.0,1,0.000000,0.0


In [16]:
utility_entropy["utility_score"] = utility_entropy["utility_score"].clip(lower=0)

mlb_roles = mlb_roles.merge(
    utility_entropy[["player_id", "season", "utility_score"]],
    on=["player_id", "season"],
    how="left"
)

mlb_roles.to_csv("mlb_roles_with_utility.csv", index=False)
utility_entropy.to_csv("season_utility_scores.csv", index=False)